In [13]:
#uncomment if not installed
#pip install yfinance
import yfinance as yf
import math
import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as rd

In [37]:
ticker = yf.Ticker("AAPL")
options_dates = ticker.options
test = np.array(list(options_dates))
option_dates1 = [dt.strptime(date, "%Y-%m-%d") for date in test]
T = (rd(dt.strptime("2024-08-15", '%Y-%m-%d'), dt.today()).months)*30/365
T
#dt.today()
#dt.strptime("2024-08-15", '%Y-%m-%d')
#dt.datetime.strptime(test, "%y-%m-%d")
#for option in options_dates:
    #dt.datetime.strptime(option, "%y-%m-%d")
#options_dates
#options_dates.index()
#options_dates
#next two lines grabs all expiery dates from ticker and matches closest one to the expiry we want
#option_dates1 = [dt.datetime.strptime(date, "%y-%m-%d") for date in options_dates]
#option_date = min(option_dates1, key = lambda x: abs(x - dt.datetime.strptime('2024-08-15', '%y-%m-%d')))
#option = ticker.option_chain(option_date)
#option

0.6575342465753424

In [38]:
#Grab a stock for our paths, S ,T, K
def grabstock(symbol, exp_date, strike,type):
    # symbol: Ticker for the option
    # exp_date: The date of expiration for the option
    # strike: Strike price
    # type: Whether a call or put
    try:
        # Fetch the stock option data
        ticker = yf.Ticker(symbol)
        options = ticker.options
        options_dates = np.array(list(options))
        #next two lines grabs all expiery dates from ticker and matches closest one to the expiry we want
        option_dates1 = [dt.strptime(date, '%Y-%m-%d') for date in options_dates]
        option_date = min(option_dates1, key = lambda x: abs(x - dt.strptime(exp_date , '%Y-%m-%d')))
        option = ticker.option_chain(option_date.strftime('%Y-%m-%d'))
        closest_strike = min(option.calls.strike, key=lambda x:abs(x-strike))
        T = (rd(option_date, dt.today()).months)*30/365
        if(type == 'call'):
            call = option.calls.loc[option.calls.strike == closest_strike].head(1)
            K = call.strike
            price = call.lastPrice
            sigma = call.impliedVolatility
            S = ticker.history(period="1d")["Close"].iloc[-1]
            r = 0.05
            q = 0.02
        elif(type == 'put'):
            put = option.puts.loc[option.puts.strike == closest_strike].head(1)
            K = put.strike
            price = put.lastPrice
            sigma = put.impliedVolatility
            S = ticker.history(period="1d")["Close"].iloc[-1]
            r = 0.05
            q = 0.02
        else:
            return "Wrong type please enter put or call when calling grabstock"
        vals = [S, T, r, q, sigma, K]
        return vals
        
    except Exception as e:
        print(f"Error: {e}")
        return None

test = grabstock("AAPL", "2024-07-15", 153, 'call')
test[1]

0.5753424657534246

In [ ]:
def paths(S, T, r, q, sigma, steps, N):
    """
    Inputs
    #S = Current stock Price
    #K = Strike Price
    #T = Time to maturity 1 year = 1, 1 months = 1/12
    #r = risk free interest rate
    #q = dividend yield
    # sigma = volatility 
    
    Output
    # [steps,N] Matrix of asset paths 
    """
    dt = T/steps
    #S_{T} = ln(S_{0})+\int_{0}^T(\mu-\frac{\sigma^2}{2})dt+\int_{0}^T \sigma dW(t)
    ST = np.log(S) +  np.cumsum(((r - q - sigma**2/2)*dt +\
                              sigma*np.sqrt(dt) * \
                              np.random.normal(size=(steps,N))),axis=0)
    
    return np.exp(ST)


steps = 100 # time steps
N = 1000 # number of trials

STs = paths(test[0],T,r, q,sigma,steps,N)
plt.plot(STs);

In [ ]:
payoffs = np.maximum(STs[-1]-K, 0)
option_price = np.mean(payoffs)*np.exp(-r*T)
option_price